In [4]:

tkn0 = db.c.w3.toChecksumAddress('0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599')
tkn1 = db.c.w3.toChecksumAddress('0x6B175474E89094C44Da98b954EedeAC495271d0F')
db.carbon_controller.strategiesByPair(tkn0, tkn1, 0, 500)

ValueError: execution reverted

In [11]:
bot.c.w3.provider.endpoint_uri


'https://rpc.tenderly.co/fork/045fdd40-5228-487a-a4b6-a513daadb2ad'

In [13]:
import web3
from fastlane_bot.data import abi as abis

carbon_controller = bot.c.w3.eth.contract(address=db.c.CARBON_CONTROLLER_ADDRESS, abi=abis.CARBON_CONTROLLER_ABI)

carbon_controller.address

'0xC537e898CD774e2dCBa3B14Ea6f34C93d5eA45e1'

In [32]:
combined_tables = pd.read_csv('fastlane_bot/data/combined_tables_new.csv')
combined_tables_old = pd.read_csv('fastlane_bot/data/combined_tables.csv')
combined_tables_old = combined_tables_old.rename({'exchange': 'exchange_name'}, axis=1)
combined_tables_with_anchor = combined_tables.merge(combined_tables_old[['address', 'exchange_name', 'fee', 'anchor']], on=['address', 'exchange_name', 'fee'], how='left')

combined_tables_with_anchor

,cid,last_updated,last_updated_block,address,exchange_name,exchange_id,fee,fee_float,pair_name,tkn0_address,...,z_0,y_0,A_0,B_0,z_1,y_1,A_1,B_1,descr,anchor
0,00125d26-4f9d-4936-9a46-7e7708cee9b5,2023-05-06 19:04:10.202420,17203475,0x7BeA39867e4169DBe237d55C8242a8f2fcDcc387,uniswap_v3,4,10000.000,0.010,USDC-eB48/WETH-6Cc2,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uniswap_v3 USDC-eB48/WETH-6Cc2 10000,NaN
1,0019a82b-c631-4e22-a855-f6f5c4407f2f,2023-05-06 19:12:55.459658,17203475,0xE8cF4874d365f3d8f66284679Cec2286a51b1128,uniswap_v3,4,3000.000,0.003,WETH-6Cc2/CADC-2Eef,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uniswap_v3 WETH-6Cc2/CADC-2Eef 3000,NaN
2,002cb90e-bf7a-4a89-b08d-c9355b3fa636,2023-05-06 19:42:10.030631,17203475,0x64F0bc5141B18B8C816dA7E7f1f26676e10506bb,uniswap_v2,3,0.003,0.003,WETH-6Cc2/MATRIX-1C36,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uniswap_v2 WETH-6Cc2/MATRIX-1C36 0.003,NaN
3,00304cc4-3510-465e-aa0a-27ff06e7cdfa,2023-05-06 19:31:14.670126,17203475,0x430Ba84FadF427eE5E8d4d78538b64C1E7456020,uniswap_v2,3,0.003,0.003,SAN-f098/WETH-6Cc2,0x7C5A0CE9267ED19B22F8cae653F198e3E8daf098,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uniswap_v2 SAN-f098/WETH-6Cc2 0.003,NaN
4,0063f6a6-f1c0-45be-9bf3-126f1dfe1fa0,2023-05-06 19:46:19.386888,17203475,0x6B1455E27902CA89eE3ABB0673A8Aa9Ce1609952,uniswap_v2,3,0.003,0.003,WETH-6Cc2/PPDEX-0cf7,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uniswap_v2 WETH-6Cc2/PPDEX-0cf7 0.003,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3474,ff1fabe2-f711-46b3-ab32-04086e625390,2023-05-06 19:40:44.385050,17203475,0xfc221e168dC56cda3dEEa3108760df13766280B6,uniswap_v2,3,0.003,0.003,SPHRI-b3Ec/WETH-6Cc2,0x8A0cdfaB62eD35b836DC0633482798421C81b3Ec,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uniswap_v2 SPHRI-b3Ec/WETH-6Cc2 0.003,NaN
3475,ff2e48f9-0df7-4a3b-a508-2d0aa33052c2,2023-05-06 19:31:02.591112,17203475,0x755C1a8F71f4210CD7B60b9439451EfCbeBa33D1,uniswap_v2,3,0.003,0.003,LPT-B239/WETH-6Cc2,0x58b6A8A3302369DAEc383334672404Ee733aB239,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uniswap_v2 LPT-B239/WETH-6Cc2 0.003,NaN
3476,ff3dc2f5-5462-4de1-9a4d-bb1df0b78f6a,2023-05-06 19:30:15.370667,17203475,0x51fe8E8FA3F9F10288fA8c2Aff1400F887D21c42,uniswap_v2,3,0.003,0.003,MITx-1Db6/WETH-6Cc2,0x4a527d8fc13C5203AB24BA0944F4Cb14658D1Db6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uniswap_v2 MITx-1Db6/WETH-6Cc2 0.003,NaN
3477,ff82edd0-a1cf-4302-9f4f-ecdeea936baa,2023-05-06 19:23:54.829193,17203475,0x3ED96D54BE53868eDbc3Ad5CCc4995710d187dC4,uniswap_v3,4,3000.000,0.003,APE-4381/USDT-1ec7,0x4d224452801ACEd8B2F0aebE155379bb5D594381,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uniswap_v3 APE-4381/USDT-1ec7 3000,NaN


In [34]:
combined_tables_with_anchor

,cid,last_updated,last_updated_block,address,exchange_name,exchange_id,fee,fee_float,pair_name,tkn0_address,...,z_0,y_0,A_0,B_0,z_1,y_1,A_1,B_1,descr,anchor
0,00125d26-4f9d-4936-9a46-7e7708cee9b5,2023-05-06 19:04:10.202420,17203475,0x7BeA39867e4169DBe237d55C8242a8f2fcDcc387,uniswap_v3,4,10000.000,0.010,USDC-eB48/WETH-6Cc2,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uniswap_v3 USDC-eB48/WETH-6Cc2 10000,NaN
1,0019a82b-c631-4e22-a855-f6f5c4407f2f,2023-05-06 19:12:55.459658,17203475,0xE8cF4874d365f3d8f66284679Cec2286a51b1128,uniswap_v3,4,3000.000,0.003,WETH-6Cc2/CADC-2Eef,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uniswap_v3 WETH-6Cc2/CADC-2Eef 3000,NaN
2,002cb90e-bf7a-4a89-b08d-c9355b3fa636,2023-05-06 19:42:10.030631,17203475,0x64F0bc5141B18B8C816dA7E7f1f26676e10506bb,uniswap_v2,3,0.003,0.003,WETH-6Cc2/MATRIX-1C36,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uniswap_v2 WETH-6Cc2/MATRIX-1C36 0.003,NaN
3,00304cc4-3510-465e-aa0a-27ff06e7cdfa,2023-05-06 19:31:14.670126,17203475,0x430Ba84FadF427eE5E8d4d78538b64C1E7456020,uniswap_v2,3,0.003,0.003,SAN-f098/WETH-6Cc2,0x7C5A0CE9267ED19B22F8cae653F198e3E8daf098,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uniswap_v2 SAN-f098/WETH-6Cc2 0.003,NaN
4,0063f6a6-f1c0-45be-9bf3-126f1dfe1fa0,2023-05-06 19:46:19.386888,17203475,0x6B1455E27902CA89eE3ABB0673A8Aa9Ce1609952,uniswap_v2,3,0.003,0.003,WETH-6Cc2/PPDEX-0cf7,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uniswap_v2 WETH-6Cc2/PPDEX-0cf7 0.003,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3474,ff1fabe2-f711-46b3-ab32-04086e625390,2023-05-06 19:40:44.385050,17203475,0xfc221e168dC56cda3dEEa3108760df13766280B6,uniswap_v2,3,0.003,0.003,SPHRI-b3Ec/WETH-6Cc2,0x8A0cdfaB62eD35b836DC0633482798421C81b3Ec,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uniswap_v2 SPHRI-b3Ec/WETH-6Cc2 0.003,NaN
3475,ff2e48f9-0df7-4a3b-a508-2d0aa33052c2,2023-05-06 19:31:02.591112,17203475,0x755C1a8F71f4210CD7B60b9439451EfCbeBa33D1,uniswap_v2,3,0.003,0.003,LPT-B239/WETH-6Cc2,0x58b6A8A3302369DAEc383334672404Ee733aB239,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uniswap_v2 LPT-B239/WETH-6Cc2 0.003,NaN
3476,ff3dc2f5-5462-4de1-9a4d-bb1df0b78f6a,2023-05-06 19:30:15.370667,17203475,0x51fe8E8FA3F9F10288fA8c2Aff1400F887D21c42,uniswap_v2,3,0.003,0.003,MITx-1Db6/WETH-6Cc2,0x4a527d8fc13C5203AB24BA0944F4Cb14658D1Db6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uniswap_v2 MITx-1Db6/WETH-6Cc2 0.003,NaN
3477,ff82edd0-a1cf-4302-9f4f-ecdeea936baa,2023-05-06 19:23:54.829193,17203475,0x3ED96D54BE53868eDbc3Ad5CCc4995710d187dC4,uniswap_v3,4,3000.000,0.003,APE-4381/USDT-1ec7,0x4d224452801ACEd8B2F0aebE155379bb5D594381,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uniswap_v3 APE-4381/USDT-1ec7 3000,NaN


In [33]:
combined_tables_with_anchor.to_csv('fastlane_bot/data/combined_tables_new.csv', index=False)

In [23]:
combined_tables_old['anchor'].unique()

array([nan])

In [18]:
def update_dictionary(my_dict):
    my_dict['new_key'] = 'new_value'

example_dict = {'original_key': 'original_value'}
update_dictionary(example_dict)

print(example_dict)

{'original_key': 'original_value', 'new_key': 'new_value'}


In [17]:
carbon_controller.functions.pairs().call()

[['0x6B175474E89094C44Da98b954EedeAC495271d0F',
  '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE'],
 ['0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84',
  '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE'],
 ['0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
  '0xdAC17F958D2ee523a2206206994597C13D831ec7'],
 ['0x6B175474E89094C44Da98b954EedeAC495271d0F',
  '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'],
 ['0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
  '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE'],
 ['0x6B175474E89094C44Da98b954EedeAC495271d0F',
  '0xdAC17F958D2ee523a2206206994597C13D831ec7'],
 ['0x1F573D6Fb3F13d689FF844B4cE37794d79a7FF1C',
  '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'],
 ['0xdAC17F958D2ee523a2206206994597C13D831ec7',
  '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE'],
 ['0xae78736Cd615f374D3085123A210448E74Fc6393',
  '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE'],
 ['0x6982508145454Ce325dDbE47a25d4ec3d2311933',
  '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'],
 ['0x1F573D6Fb3F13d689FF844B4c

In [14]:
for pair in carbon_controller.pairs():
    print(tkn1==pair[0], tkn0==pair[1])

AttributeError: 'Contract' object has no attribute 'pairs'

In [5]:
db.carbon_controller.pairs()

(('0x6B175474E89094C44Da98b954EedeAC495271d0F',
  '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE'),
 ('0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84',
  '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE'),
 ('0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
  '0xdAC17F958D2ee523a2206206994597C13D831ec7'),
 ('0x6B175474E89094C44Da98b954EedeAC495271d0F',
  '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'),
 ('0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
  '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE'),
 ('0x6B175474E89094C44Da98b954EedeAC495271d0F',
  '0xdAC17F958D2ee523a2206206994597C13D831ec7'),
 ('0x1F573D6Fb3F13d689FF844B4cE37794d79a7FF1C',
  '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'),
 ('0xdAC17F958D2ee523a2206206994597C13D831ec7',
  '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE'),
 ('0xae78736Cd615f374D3085123A210448E74Fc6393',
  '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE'),
 ('0x6982508145454Ce325dDbE47a25d4ec3d2311933',
  '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'),
 ('0x1F573D6Fb3F13d689FF844B4c

In [3]:
bot.c.w3.provider.endpoint_uri

'https://rpc.tenderly.co/fork/045fdd40-5228-487a-a4b6-a513daadb2ad'

In [1]:
import numpy as np
import pandas as pd
from fastlane_bot.bot import CarbonBot
from fastlane_bot.config import Config

pools_and_token_table = pd.read_csv('/Users/mikewcasale/Local/projects/bancor/carbonbot-main/fastlane_bot/data/combined_tables.csv', low_memory=False).drop('id', axis=1)

cfg = Config.new(config=Config.CONFIG_TENDERLY)
bot = CarbonBot(ConfigObj=cfg)

db = bot.db

db.c.w3.provider.endpoint_uri

Using default database url, if you want to use a different database, set the backend_url found at the bottom of manager_base.py
Error adding Ethereum blockchain to database Ethereum, (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "ix_blockchains_name"
DETAIL:  Key (name)=(Ethereum) already exists.

[SQL: INSERT INTO blockchains (name, block_number) VALUES (%(name)s, %(block_number)s) RETURNING blockchains.id]
[parameters: {'name': 'Ethereum', 'block_number': 0}]
(Background on this error at: https://sqlalche.me/e/20/gkpj) skipping...
Error adding exchange carbon_v1 to database, (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "exchanges_pkey"
DETAIL:  Key (id)=(6) already exists.

[SQL: INSERT INTO exchanges (id, name, blockchain_name) VALUES (%(id)s, %(name)s, %(blockchain_name)s)]
[parameters: {'id': 6, 'name': 'carbon_v1', 'blockchain_name': 'Ethereum'}]
(Background on this error at: https://sqlalche.me/e/20/gkpj) ski

'https://rpc.tenderly.co/fork/045fdd40-5228-487a-a4b6-a513daadb2ad'

In [11]:
import numpy as np
import pandas as pd
from fastlane_bot.bot import CarbonBot
from fastlane_bot.config import Config

pools_and_token_table = pd.read_csv('/Users/mikewcasale/Local/projects/bancor/carbonbot-main/fastlane_bot/data/combined_tables.csv', low_memory=False).drop('id', axis=1)

cfg = Config.new(config=Config.CONFIG_TENDERLY)
bot = CarbonBot(ConfigObj=cfg)

db = bot.db

carbon_pairs = db.get_carbon_pairs()
for pair in carbon_pairs:

    tkn0 = db.get_or_create_token(address=pair[0])
    tkn1 = db.get_or_create_token(address=pair[1])
    pair_name = f"{tkn0.key}/{tkn1.key}"
    if pair_name not in pools_and_token_table['pair_name'].unique().tolist():
        # bypairs.append(pair_name)
        print(f"Added {pair_name} to bypairs list")

        dbrow = pd.DataFrame({'pair_name': [pair_name], 'tkn0_address': [tkn0.address], 'tkn1_address': [tkn1.address], 'exchange_name': ['carbon_v1'], 'address': db.carbon_controller.address})
        missing_cols = [col for col in pools_and_token_table.columns if col not in dbrow.columns]
        for col in missing_cols:
            dbrow[col] = [np.NaN]
        pools_and_token_table = pools_and_token_table.append(dbrow, ignore_index=True)

pools_and_token_table


Added DAI-1d0F/ETH-EEeE to bypairs list


/var/folders/rt/qnj8r6yd6131ccxkw_k9d9gc0000gn/T/ipykernel_97221/175557020.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pools_and_token_table = pools_and_token_table.append(dbrow, ignore_index=True)


Added USDC-eB48/ETH-EEeE to bypairs list


/var/folders/rt/qnj8r6yd6131ccxkw_k9d9gc0000gn/T/ipykernel_97221/175557020.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pools_and_token_table = pools_and_token_table.append(dbrow, ignore_index=True)


Added USDT-1ec7/ETH-EEeE to bypairs list


...

Added USDC-eB48/JAY-CA6E to bypairs list


/var/folders/rt/qnj8r6yd6131ccxkw_k9d9gc0000gn/T/ipykernel_97221/175557020.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pools_and_token_table = pools_and_token_table.append(dbrow, ignore_index=True)
2023-05-07 08:45:36,193 [fastlane:INFO] - [model_managers.Token] Successfully created token: WYD-8991
2023-05-07 08:45:36,193 [fastlane:INFO] - [model_managers.Token] Successfully created token: WYD-8991
2023-05-07 08:45:37,746 [fastlane:INFO] - [model_managers.Token] Successfully created token: WSB-0c49
2023-05-07 08:45:37,746 [fastlane:INFO] - [model_managers.Token] Successfully created token: WSB-0c49


Added EWTB-6054/ETH-EEeE to bypairs list


/var/folders/rt/qnj8r6yd6131ccxkw_k9d9gc0000gn/T/ipykernel_97221/175557020.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pools_and_token_table = pools_and_token_table.append(dbrow, ignore_index=True)


,cid,last_updated,last_updated_block,descr,pair_name,exchange_name,fee,fee_float,tkn0_balance,tkn1_balance,...,tkn0_address,tkn1_address,tkn0_key,tkn1_key,tkn0_decimals,tkn1_decimals,exchange_id,tkn0_symbol,tkn1_symbol,anchor
0,340282366920938463463374607431768211457,2023-05-03 17:58:01.525631,17181818.0,carbon_v1 ETH-EEeE/DAI-1d0F None,ETH-EEeE/DAI-1d0F,carbon_v1,0.0,0.0,NaN,NaN,...,0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE,0x6B175474E89094C44Da98b954EedeAC495271d0F,ETH-EEeE,DAI-1d0F,18.0,18.0,6.0,ETH,DAI,NaN
1,680564733841876926926749214863536422914,2023-05-03 17:58:02.284832,17181818.0,carbon_v1 ETH-EEeE/stETH-fE84 None,ETH-EEeE/stETH-fE84,carbon_v1,0.0,0.0,NaN,NaN,...,0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE,0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84,ETH-EEeE,stETH-fE84,18.0,18.0,6.0,ETH,stETH,NaN
2,680564733841876926926749214863536422967,2023-05-03 17:58:02.297098,17181818.0,carbon_v1 stETH-fE84/ETH-EEeE None,stETH-fE84/ETH-EEeE,carbon_v1,0.0,0.0,NaN,NaN,...,0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84,0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE,stETH-fE84,ETH-EEeE,18.0,18.0,6.0,stETH,ETH,NaN
3,1020847100762815390390123822295304634371,2023-05-03 17:58:03.773714,17181818.0,carbon_v1 USDC-eB48/USDT-1ec7 None,USDC-eB48/USDT-1ec7,carbon_v1,0.0,0.0,NaN,NaN,...,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0xdAC17F958D2ee523a2206206994597C13D831ec7,USDC-eB48,USDT-1ec7,6.0,6.0,6.0,USDC,USDT,NaN
4,1020847100762815390390123822295304634391,2023-05-03 17:58:03.781903,17181818.0,carbon_v1 USDC-eB48/USDT-1ec7 None,USDC-eB48/USDT-1ec7,carbon_v1,0.0,0.0,NaN,NaN,...,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0xdAC17F958D2ee523a2206206994597C13D831ec7,USDC-eB48,USDT-1ec7,6.0,6.0,6.0,USDC,USDT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
844,NaN,NaN,NaN,NaN,RAIL-A33D/ETH-EEeE,carbon_v1,NaN,NaN,NaN,NaN,...,0xe76C6c83af64e4C60245D8C7dE953DF673a7A33D,0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
845,NaN,NaN,NaN,NaN,WBTC-C599/ETH-EEeE,carbon_v1,NaN,NaN,NaN,NaN,...,0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599,0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
846,NaN,NaN,NaN,NaN,USDC-eB48/pepecoin-489A,carbon_v1,NaN,NaN,NaN,NaN,...,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0xA9E8aCf069C58aEc8825542845Fd754e41a9489A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
847,NaN,NaN,NaN,NaN,USDC-eB48/JAY-CA6E,carbon_v1,NaN,NaN,NaN,NaN,...,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0xDA7C0810cE6F8329786160bb3d1734cf6661CA6E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
import pandas as pd

df = pd.read_csv('fastlane_bot/data/combined_tables.csv')
df

,id,cid,last_updated,last_updated_block,descr,pair_name,exchange_name,fee,fee_float,tkn0_balance,...,tkn0_address,tkn1_address,tkn0_key,tkn1_key,tkn0_decimals,tkn1_decimals,exchange_id,tkn0_symbol,tkn1_symbol,anchor
0,1,340282366920938463463374607431768211457,2023-05-03 17:58:01.525631,17181818,carbon_v1 ETH-EEeE/DAI-1d0F None,ETH-EEeE/DAI-1d0F,carbon_v1,0.000,0.000,NaN,...,0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE,0x6B175474E89094C44Da98b954EedeAC495271d0F,ETH-EEeE,DAI-1d0F,18,18,6,ETH,DAI,NaN
1,2,680564733841876926926749214863536422914,2023-05-03 17:58:02.284832,17181818,carbon_v1 ETH-EEeE/stETH-fE84 None,ETH-EEeE/stETH-fE84,carbon_v1,0.000,0.000,NaN,...,0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE,0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84,ETH-EEeE,stETH-fE84,18,18,6,ETH,stETH,NaN
2,3,680564733841876926926749214863536422967,2023-05-03 17:58:02.297098,17181818,carbon_v1 stETH-fE84/ETH-EEeE None,stETH-fE84/ETH-EEeE,carbon_v1,0.000,0.000,NaN,...,0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84,0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE,stETH-fE84,ETH-EEeE,18,18,6,stETH,ETH,NaN
3,4,1020847100762815390390123822295304634371,2023-05-03 17:58:03.773714,17181818,carbon_v1 USDC-eB48/USDT-1ec7 None,USDC-eB48/USDT-1ec7,carbon_v1,0.000,0.000,NaN,...,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0xdAC17F958D2ee523a2206206994597C13D831ec7,USDC-eB48,USDT-1ec7,6,6,6,USDC,USDT,NaN
4,5,1020847100762815390390123822295304634391,2023-05-03 17:58:03.781903,17181818,carbon_v1 USDC-eB48/USDT-1ec7 None,USDC-eB48/USDT-1ec7,carbon_v1,0.000,0.000,NaN,...,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0xdAC17F958D2ee523a2206206994597C13D831ec7,USDC-eB48,USDT-1ec7,6,6,6,USDC,USDT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,836,836,2023-05-03 18:17:00.341882,17181912,sushiswap_v2 DAI-1d0F/RAIL-A33D 0.003,DAI-1d0F/RAIL-A33D,sushiswap_v2,0.003,0.003,40780567355567253940127,...,0x6B175474E89094C44Da98b954EedeAC495271d0F,0xe76C6c83af64e4C60245D8C7dE953DF673a7A33D,DAI-1d0F,RAIL-A33D,18,18,5,DAI,RAIL,NaN
836,837,837,2023-05-03 18:17:01.151991,17181912,sushiswap_v2 USDC-eB48/RAIL-A33D 0.003,USDC-eB48/RAIL-A33D,sushiswap_v2,0.003,0.003,115699,...,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0xe76C6c83af64e4C60245D8C7dE953DF673a7A33D,USDC-eB48,RAIL-A33D,6,18,5,USDC,RAIL,NaN
837,838,838,2023-05-03 18:17:01.960506,17181913,sushiswap_v2 WETH-6Cc2/RAIL-A33D 0.003,WETH-6Cc2/RAIL-A33D,sushiswap_v2,0.003,0.003,123492287181744769557,...,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xe76C6c83af64e4C60245D8C7dE953DF673a7A33D,WETH-6Cc2,RAIL-A33D,18,18,5,WETH,RAIL,NaN
838,839,839,2023-05-03 18:17:02.770204,17181913,sushiswap_v2 DAI-1d0F/USDC-eB48 0.003,DAI-1d0F/USDC-eB48,sushiswap_v2,0.003,0.003,12842495708805333028991,...,0x6B175474E89094C44Da98b954EedeAC495271d0F,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,DAI-1d0F,USDC-eB48,18,6,5,DAI,USDC,NaN


In [1]:
import pandas as pd
from decimal import Decimal
from typing import List, Dict, Any
from fastlane_bot.tools.optimizer import CPCArbOptimizer
from dataclasses import asdict
from fastlane_bot.helpers import TradeInstruction

In [2]:
## mock only
best_trade_instructions_dic = [
{'cid': '218',
  'tknin': '0x0-1AD5',
  'amtin': 5309.369973833374,
  'tknout': 'WETH-6Cc2',
  'amtout': -0.09179346660459942},
{'cid': '9868188640707215440437863615521278132277-1',
  'tknin': 'WETH-6Cc2',
  'amtin': 0.0785107723399976,
  'tknout': '0x0-1AD5',
  'amtout': -5363.0},
{'cid': '9868188640707215440437863615521278132278-1',
  'tknin': 'WETH-6Cc2',
  'amtin': 0.0111111111111111,
  'tknout': '0x0-1AD5',
  'amtout': -3000.0},
{'cid': '219',
  'tknin': '0x0-1AD5',
  'amtin': 5309.369973833374,
  'tknout': 'WETH-6Cc2',
  'amtout': -0.09179346660459942},
{'cid': '9868188640707215440437863615521278132280-0',
  'tknin': '0x0-1AD5',
  'amtin': 1000.0,
  'tknout': 'WETH-6Cc2',
  'amtout': -0.001},
{'cid': '9868188640707215440437863615521278132281-0',
  'tknin': '0x0-1AD5',
  'amtin':  2000,
  'tknout': 'WETH-6Cc2',
  'amtout': -0.05},
  {'cid': '220',
  'tknin': '0x0-1AD5',
  'amtin': 5309.369973833374,
  'tknout': 'WETH-6Cc2',
  'amtout': -0.09179346660459942},
{'cid': '9868188640707215440437863615521278132290-1',
  'tknin': 'WETH-6Cc2',
  'amtin': 0.016351323399976,
  'tknout': '0x0-1AD5',
  'amtout': -1234.0},
{'cid': '9868188640707215440437863615521278132291-1',
  'tknin': 'WETH-6Cc2',
  'amtin': 0.022222222222,
  'tknout': '0x0-1AD5',
  'amtout': -2000.0},
]
best_src_token = 'WETH-6Cc2'

In [3]:
def simple_ordering_by_src_token(best_trade_instructions_dic, best_src_token):
    '''
    Reorders a trade_instructions_dct so that all items where the best_src_token is the tknin are before others
    '''
    ordered_trade_instructions_dct = [x for x in best_trade_instructions_dic if x['tknin'] == best_src_token] + [x for x in best_trade_instructions_dic if x['tknin'] != best_src_token]

    return ordered_trade_instructions_dct

In [4]:
### TEST ###
ordered_trade_instructions_dct = simple_ordering_by_src_token(best_trade_instructions_dic, best_src_token)
srctoken_indexes = [i for i,c in list(enumerate(ordered_trade_instructions_dct)) if c['tknin'] == best_src_token]
non_srctoken_indexes = [i for i,c in list(enumerate(ordered_trade_instructions_dct)) if c['tknin'] != best_src_token]
assert max(srctoken_indexes) < min(non_srctoken_indexes)
### TEST ###

In [5]:
def basic_scaling_alternative_to_exact(best_trade_instructions_dic, best_src_token):
    '''
    For items in the trade_instruction_dic scale the amtin by 0.999 if its the src_token, else by 0.99

    NOTE: Since we update the info in the DICTIONARY, this actually voids the info in the DATAFRAME
    '''
    scaled_best_trade_instructions_dic = []
    for x in best_trade_instructions_dic:
        scaled_best_trade_instructions_dic += [{k:v for k,v in x.items()}]

    for i in range(len(scaled_best_trade_instructions_dic)):
        if scaled_best_trade_instructions_dic[i]["tknin"] == best_src_token:
            scaled_best_trade_instructions_dic[i]["amtin"] *= 0.999
        else:
            scaled_best_trade_instructions_dic[i]["amtin"] *= 0.99

    return scaled_best_trade_instructions_dic

In [6]:
### TEST ###
scaled_best_trade_instructions_dic = basic_scaling_alternative_to_exact(best_trade_instructions_dic, best_src_token)
assert sum([x['amtin'] for x in scaled_best_trade_instructions_dic]) < sum([x['amtin'] for x in best_trade_instructions_dic])
### TEST ###

In [7]:
trade_instructions_objects = [TradeInstruction(**ti) for ti in best_trade_instructions_dic]
trade_instructions_objects

[TradeInstruction(cid='218', tknin='0x0-1AD5', amtin=5309.369973833374, tknout='WETH-6Cc2', amtout=-0.09179346660459942, pair_sorting=None, raw_txs=None, custom_data=''),
 TradeInstruction(cid='9868188640707215440437863615521278132277-1', tknin='WETH-6Cc2', amtin=0.0785107723399976, tknout='0x0-1AD5', amtout=-5363.0, pair_sorting=None, raw_txs=None, custom_data=''),
 TradeInstruction(cid='9868188640707215440437863615521278132278-1', tknin='WETH-6Cc2', amtin=0.0111111111111111, tknout='0x0-1AD5', amtout=-3000.0, pair_sorting=None, raw_txs=None, custom_data=''),
 TradeInstruction(cid='219', tknin='0x0-1AD5', amtin=5309.369973833374, tknout='WETH-6Cc2', amtout=-0.09179346660459942, pair_sorting=None, raw_txs=None, custom_data=''),
 TradeInstruction(cid='9868188640707215440437863615521278132280-0', tknin='0x0-1AD5', amtin=1000.0, tknout='WETH-6Cc2', amtout=-0.001, pair_sorting=None, raw_txs=None, custom_data=''),
 TradeInstruction(cid='9868188640707215440437863615521278132281-0', tknin='0x

In [8]:
def _get_trade_dicts_from_objects(trade_instructions: List[TradeInstruction]) -> List[Dict[str, Any]]:
    return [
        {
            "cid": instr.cid + "-" + str(instr.cid_tkn)
            if instr.cid_tkn
            else instr.cid,
            "tknin": instr.tknin,
            "amtin": instr.amtin,
            "tknout": instr.tknout,
            "amtout": instr.amtout,
        }
        for instr in trade_instructions
    ]

In [9]:
def slice_dataframe(df):
    slices = []
    current_pair_sorting = df.pair_sorting.values[0]
    current_slice = []

    for index, row in df.iterrows():
        if row['pair_sorting'] == current_pair_sorting:
            current_slice.append(index)
        else:
            slices.append(df.loc[current_slice])
            current_pair_sorting = row['pair_sorting']
            current_slice = [index]

    slices.append(df.loc[current_slice])

    min_index = []
    for df in slices:
        min_index += [min(df.index.values)]

    
    return list(zip(min_index, slices))


In [10]:
def aggregate_carbon_trades(trade_instructions_objects: List[TradeInstruction]) -> List[TradeInstruction]:
    """
    Aggregate carbon independent IDs and create trade instructions.

    This function takes a list of dictionaries containing trade instructions,
    aggregates the instructions with carbon independent IDs, and creates
    a list of TradeInstruction objects.

    Parameters
    ----------
    trade_instructions : List[TradeInstruction]
        A list of trade instructions as TradeInstruction objects.

    Returns
    -------
    List[TradeInstruction]
        A list of aggregated trade instructions as TradeInstruction objects.

    """
    # Get the indices of the carbon trades
    listti = _get_trade_dicts_from_objects(trade_instructions_objects)
    df = pd.DataFrame(listti)
    df["pair_sorting"] = df.tknin + df.tknout
    df['carbon'] = [True if '-' in df.cid[i] else False for i in df.index]

    carbons = df[df['carbon']].copy()
    nocarbons = df[~df['carbon']].copy()
    nocarbons["raw_txs"] = str([])

    carbons.drop(['carbon'], axis=1, inplace=True)
    nocarbons.drop(['carbon'], axis=1, inplace=True)

    new_trade_instructions_nocarbons = {i: nocarbons.loc[i].to_dict() for i in nocarbons.index}

    result = slice_dataframe(carbons)
    new_trade_instructions_carbons = {min_index:
        {
            "pair_sorting": newdf.pair_sorting.values[0],
            "cid": newdf.cid.values[0],
            "tknin": newdf.tknin.values[0],
            "amtin": newdf.amtin.sum(),
            "tknout": newdf.tknout.values[0],
            "amtout": newdf.amtout.sum(),
            "raw_txs": str(newdf.to_dict(orient="records")),
        }
        for min_index, newdf in slice_dataframe(carbons)}

    new_trade_instructions_carbons.update(new_trade_instructions_nocarbons)
    agg_trade_instructions = []
    for i in sorted(list(new_trade_instructions_carbons.keys())):
        agg_trade_instructions += [TradeInstruction(**new_trade_instructions_carbons[i])]
    return agg_trade_instructions

In [11]:
# this example is different to the example usage in that the dict hasn't been sorted first, 
# I did this so I could verify that the aggregation works on adjacent same-pair only
agg_trade_instructions = aggregate_carbon_trades(trade_instructions_objects)
agg_trade_instructions

[TradeInstruction(cid='218', tknin='0x0-1AD5', amtin=5309.369973833374, tknout='WETH-6Cc2', amtout=-0.09179346660459942, pair_sorting='0x0-1AD5WETH-6Cc2', raw_txs='[]', custom_data=''),
 TradeInstruction(cid='9868188640707215440437863615521278132277-1', tknin='WETH-6Cc2', amtin=0.0896218834511087, tknout='0x0-1AD5', amtout=-8363.0, pair_sorting='WETH-6Cc20x0-1AD5', raw_txs="[{'cid': '9868188640707215440437863615521278132277-1', 'tknin': 'WETH-6Cc2', 'amtin': 0.0785107723399976, 'tknout': '0x0-1AD5', 'amtout': -5363.0, 'pair_sorting': 'WETH-6Cc20x0-1AD5'}, {'cid': '9868188640707215440437863615521278132278-1', 'tknin': 'WETH-6Cc2', 'amtin': 0.0111111111111111, 'tknout': '0x0-1AD5', 'amtout': -3000.0, 'pair_sorting': 'WETH-6Cc20x0-1AD5'}]", custom_data=''),
 TradeInstruction(cid='219', tknin='0x0-1AD5', amtin=5309.369973833374, tknout='WETH-6Cc2', amtout=-0.09179346660459942, pair_sorting='0x0-1AD5WETH-6Cc2', raw_txs='[]', custom_data=''),
 TradeInstruction(cid='98681886407072154404378636

# Example Usage

In [12]:
ordered_trade_instructions_dct = simple_ordering_by_src_token(best_trade_instructions_dic, best_src_token)
scaled_best_trade_instructions_dic = basic_scaling_alternative_to_exact(ordered_trade_instructions_dct, best_src_token)

# YOU DONT NEED THIS just a stand-in for _convert_trade_instructions
trade_instructions_objects = [TradeInstruction(**ti) for ti in scaled_best_trade_instructions_dic]

agg_trade_instructions = aggregate_carbon_trades(trade_instructions_objects)
agg_trade_instructions

[TradeInstruction(cid='9868188640707215440437863615521278132277-1', tknin='WETH-6Cc2', amtin=0.12806723364401162, tknout='0x0-1AD5', amtout=-11597.0, pair_sorting='WETH-6Cc20x0-1AD5', raw_txs="[{'cid': '9868188640707215440437863615521278132277-1', 'tknin': 'WETH-6Cc2', 'amtin': 0.0784322615676576, 'tknout': '0x0-1AD5', 'amtout': -5363.0, 'pair_sorting': 'WETH-6Cc20x0-1AD5'}, {'cid': '9868188640707215440437863615521278132278-1', 'tknin': 'WETH-6Cc2', 'amtin': 0.011099999999999988, 'tknout': '0x0-1AD5', 'amtout': -3000.0, 'pair_sorting': 'WETH-6Cc20x0-1AD5'}, {'cid': '9868188640707215440437863615521278132290-1', 'tknin': 'WETH-6Cc2', 'amtin': 0.016334972076576026, 'tknout': '0x0-1AD5', 'amtout': -1234.0, 'pair_sorting': 'WETH-6Cc20x0-1AD5'}, {'cid': '9868188640707215440437863615521278132291-1', 'tknin': 'WETH-6Cc2', 'amtin': 0.022199999999778, 'tknout': '0x0-1AD5', 'amtout': -2000.0, 'pair_sorting': 'WETH-6Cc20x0-1AD5'}]", custom_data=''),
 TradeInstruction(cid='218', tknin='0x0-1AD5', a